In [12]:
import cv2

# Open camera
cap = cv2.VideoCapture(0)

# Set frame size
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)


# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = None
recording = False

while True:
    # Read frame
    ret, frame = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(gray)

    # Find contours
    contours, hierarchy = cv2.findContours(fg_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Detect motion
    motion_detected = False
    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if contour_area > 600: # filter out small contours
            motion_detected = True
            break

    # Start recording if motion detected
    if motion_detected and not recording:
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))
        recording = True
        print('Recording started')

    # Stop recording if no motion detected
    if not motion_detected and recording:
        out.release()
        recording = False
        print('Recording stopped')

    # Draw bounding boxes
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        contour_area = cv2.contourArea(contour)
        if contour_area > 500: # filter out small contours
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Write frame to output file if recording
    if recording:
        out.write(frame)

    # Display frame
    cv2.imshow('Security Camera', frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
if out is not None:
    out.release()
cv2.destroyAllWindows()


Recording started
Recording stopped
Recording started
Recording stopped
Recording started
Recording stopped
Recording started
Recording stopped
Recording started
Recording stopped
Recording started
Recording stopped
Recording started
Recording stopped
